In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import os
import sys
sys.path.append(os.path.join(os.getcwd(), "src"))

import pandas as pd
import numpy as np
import seaborn as sns
import importlib
import time
import datetime
import matplotlib.pyplot as plt
from scipy import stats as scipystats

from resources import RESOURCE_PATH
from stats import sleepStats
from util import utils
from util import plotting as mplot

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.handlers[0].stream = sys.stdout

### Load and prepare data

In [ ]:
dataFolder = "..\\..\\scripts_data\\fitbit_analyzer\\dataDump"
#filepath =  RESOURCE_PATH + "\\unittest\\test_sleepStats.csv"

In [ ]:
start = time.time()
sleepData = utils.loadSleepData(dataFolder)
end = time.time()
print("Data loaded in {:.2f}s".format(end - start))
print("Loaded {} dataframes".format(len(sleepData)))
print("{} total entries".format(np.sum([df.size for df in sleepData])))
print("Sample from first dataframe:")
print(sleepData[0].head())

### Generate and save stats to file

In [ ]:
start = time.time()
basicAndTimingStats = sleepStats.generateStatsFrom(sleepData, sleepStats.STATS_NAME_BASIC_AND_TIMING)
end = time.time()
print("Computed basicAndTimingStats in {:.2f}s".format(end - start))
intervalsStats = sleepStats.generateStatsFrom(sleepData, sleepStats.STATS_NAME_INTERVALS)
end = time.time()
print("Computed intervalsStats in {:.2f}s".format(end - start))
intradayStats = sleepStats.generateStatsFrom(sleepData, sleepStats.STATS_NAME_INTRADAY)
end = time.time()
print("Computed intradayStats in {:.2f}s".format(end - start))

In [ ]:
#print(basicAndTimingStats.head())
#print(intervalsStats.head())
#print(intradayStats.head())

In [ ]:
today = datetime.date.today().strftime("%Y_%m_%d")
basicAndTimingStatsFilepath = "{}\\..\\basicAndTimingStats_{}.csv".format(dataFolder, today)
intervalsStatsFilepath = "{}\\..\\intervalStats_{}.csv".format(dataFolder, today)
intradayStatsFilepath = "{}\\..\\intradayStats_{}.csv".format(dataFolder, today)

basicAndTimingStats.reset_index().to_csv(basicAndTimingStatsFilepath, index=False)
intervalsStats.reset_index().to_csv(intervalsStatsFilepath, index=False)
intradayStats.reset_index().to_csv(intradayStatsFilepath, index=False)

### Load previously exported stats and plot

In [ ]:
exportedDate = today
stats = pd.read_csv("{}\\..\\basicAndTimingStats_{}.csv".format(dataFolder, exportedDate), parse_dates=['date'])
print(stats.head())

In [ ]:
testData = stats.set_index('date')['restless']
testData.resample('20D').mean().plot()
testData.plot()
sns.plt.show()

In [ ]:
mplot.plotYearAndMonthStatsSleep(stats)
#mplot.plotPreliminaryStats(stats)
#mplot.plotWeekdayStatsSleep(stats)
#mplot.plotDailyStatsSleep(stats)
#mplot.plotMonthlyStatsSleep(stats)

In [ ]:
intradayStats = pd.read_csv("{}\\..\\intradayStats_{}.csv".format(dataFolder, exportedDate))
#print(intradayStats.head())
intradayStats.drop("date", axis=1, inplace=True)
data = intradayStats.apply(pd.value_counts)
mplot.plotSleepValueHeatmap(data, sleepValue=1)

In [ ]:
normIntradayCountStats = sleepStats.normalizedIntradayCountStats(intradayStats)
centeredIntradayCountStats = sleepStats.centerIntradayCountStats(normIntradayCountStats)
mplot.plotSleepValueHeatmap(centeredIntradayCountStats, sleepValue=3)